In [13]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
from IPython.display import display, Markdown
from utility import YouTubeAPI_utility as api, Preprocess_EDA as eda


df = eda.integrate_clean_data()

us_df = df[(df['country']=='US')]
gb_df = df[(df['country']=='GB')]

display(us_df)

#us_df = pd.read_csv("../data/US_youtube_trending_data.csv")
#gb_df = pd.read_csv("../data/GB_youtube_trending_data.csv")

In [12]:
#publishAt -> publish date, publish weekday(.isoweekday()) and publish time
def getWeekdayTxt(date):
    
    weekday=""
    if date.isoweekday()== 1:
        weekday = "Monday"
    elif date.isoweekday()== 2:
        weekday = "Tuesday"    
    elif date.isoweekday()== 3:
        weekday = "Wednesday"
    elif date.isoweekday()== 4:
        weekday = "Thursday"
    elif date.isoweekday()== 5:
        weekday = "Friday"
    elif date.isoweekday()== 6:
        weekday = "Saturday"
    elif date.isoweekday()== 7:
        weekday = "Sunday"
    
    return weekday

def find_number(text):
    
    num = re.findall(r'[0-9]+',text)
    
    return "".join(num)


#Analysis from trending date >= 2021-01-01
us_df['trending_date'] = us_df['trending_date'].apply(lambda x: find_number(x))
us_df['trending_date'] = pd.to_datetime(us_df['trending_date'], format='%Y%m%d%H%M%S')
us_df = us_df[(us_df['trending_date']>='2021-01-01')]
us_df = us_df.sort_values(by=['trending_date'], ascending=True)
#breakdown publishAt datetime elements
us_df = us_df.sort_values(by=['publishedAt'], ascending=False)
us_df['publishedAt'] = us_df['publishedAt'].apply(lambda x: find_number(x))
us_df['publishedAt_date'] = us_df['publishedAt'].str.slice(0, 8)
us_df['publishedAt_date'] = pd.to_datetime(us_df['publishedAt_date'], format='%Y%m%d')
us_df['publishedAt_weekday'] = us_df['publishedAt_date'].apply(lambda x: getWeekdayTxt(x))
us_df['publishedAt_time'] = us_df['publishedAt'].str.slice(8, 14)
us_df['publishedAt_hour'] = pd.to_datetime(us_df['publishedAt_time'], format='%H%M%S').dt.strftime('%I %p')
us_df['publishedAt_time'] = pd.to_datetime(us_df['publishedAt_time'], format='%H%M%S').dt.time

display(us_df)

**publish date and publish time related on trending videos list**

In [ ]:
weekday_us_df =  us_df[['video_id','publishedAt_weekday']].groupby('publishedAt_weekday', as_index=False).count()
weekday_us_df['publishedAt_weekday']=pd.Categorical(weekday_us_df['publishedAt_weekday'], ["Monday", "Tuesday", "Wednesday","Thursday","Friday","Saturday","Sunday"])
weekday_us_df = weekday_us_df.sort_values('publishedAt_weekday')
display(weekday_us_df)
weekday_us_trending_graph = weekday_us_df.plot.bar(x='publishedAt_weekday', y='video_id', figsize=(15,5), title='Number Of Trending Video Published Day of Week', alpha=0.5)      

In [ ]:
hour_us_df =  us_df[['video_id','publishedAt_hour']].groupby('publishedAt_hour', as_index=False).count()
hour_us_df['publishedAt_hour']=pd.Categorical(hour_us_df['publishedAt_hour'], ["12 AM", "01 AM", "02 AM","03 AM","04 AM","05 AM","06 AM","07 AM","08 AM","09 AM","10 AM","11 AM",
                                                                              "12 PM", "01 PM", "02 PM","03 PM","04 PM","05 PM","06 PM","07 PM","08 PM","09 PM","10 PM","11 PM"])
hour_us_df = hour_us_df.sort_values('publishedAt_hour')
display(hour_us_df)
hour_us_trending_graph = hour_us_df.plot.bar(x='publishedAt_hour', y='video_id', figsize=(15,5), title='Number Of Trending Video Published Hour', alpha=0.5)      

In [ ]:
weekday_hour_us_df = us_df[['video_id','publishedAt_weekday','publishedAt_hour']].groupby(['publishedAt_weekday','publishedAt_hour'], as_index=False).count()
weekday_hour_us_df['publishedAt_weekday']=pd.Categorical(weekday_hour_us_df['publishedAt_weekday'], ["Monday", "Tuesday", "Wednesday","Thursday","Friday","Saturday","Sunday"])
weekday_hour_us_df['publishedAt_hour']=pd.Categorical(weekday_hour_us_df['publishedAt_hour'], ["12 AM", "01 AM", "02 AM","03 AM","04 AM","05 AM","06 AM","07 AM","08 AM","09 AM","10 AM","11 AM",
                                                                                                 "12 PM", "01 PM", "02 PM","03 PM","04 PM","05 PM","06 PM","07 PM","08 PM","09 PM","10 PM","11 PM"])
weekday_hour_us_df = weekday_hour_us_df.sort_values(['publishedAt_weekday','publishedAt_hour'])
weekday_hour_us_df["publishedAt_weekday_hour"] = weekday_hour_us_df["publishedAt_weekday"].astype(str)+ " " + weekday_hour_us_df["publishedAt_hour"].astype(str)
weekday_hour_us_trending_graph = weekday_hour_us_df.plot.bar(x='publishedAt_weekday_hour', y='video_id', figsize=(40,10), title='Number Of Trending Video Published Weekday & Hour', alpha=0.5)  
weekday_hour_us_df = weekday_hour_us_df.sort_values('video_id', ascending=False)
top_10_weekday_hour_us_df = weekday_hour_us_df.head(10)
top_10_weekday_hour_us_trending_graph = top_10_weekday_hour_us_df.plot.bar(x='publishedAt_weekday_hour', y='video_id', figsize=(15,5), title='Top 10 Published Weekday & Hour of Trending Videos', alpha=0.5)  

In [ ]:
#duplicate video id
duplicate_video_us_df = us_df[['video_id','view_count']].groupby(['video_id'], as_index=False).count()
duplicate_video_us_df = duplicate_video_us_df.rename(columns={"view_count":"count"})
duplicate_video_us_df = duplicate_video_us_df[(duplicate_video_us_df['count']>1)]
display(duplicate_video_us_df.sort_values('count', ascending=True))

In [ ]:
#wwB91-jxCK8
check_duplicate_df = us_df[(us_df['video_id']=='wwB91-jxCK8')]
display(check_duplicate_df)